In [1]:
import gensim
from xlrd import open_workbook
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import json
import pandas as pd
from nltk.corpus import stopwords

import nltk
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from os import listdir, makedirs
from os.path import isfile, join, exists
import os
from nltk.corpus import stopwords
from xlrd import open_workbook
import json
from gensim import similarities
import numpy as np

In [2]:
"""
book = open_workbook('UNSPSC English v220601 project.xlsx')
'''book = open_workbook('Unspec List2b.xlsx')'''
'''To work on the UNSPSC sheet you need to change the values of 0 to 12 and 1 to
16 in order to make the it work.'''
dict_list = []
sheet = book.sheet_by_index(0)
# read header values into the list
keys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]

for row_index in range(1, sheet.nrows):
    d = {keys[col_index]: sheet.cell(row_index, col_index).value
         for col_index in range(sheet.ncols)}
    dict_list.append(d)

print(len(dict_list))
    
doclist =[]
listOfEntry =[]
entrylist = []
df7 = pd.DataFrame()


for entry in dict_list:
    if(entry.get("Family")==""):
        continue
    elif(entry.get("Family")!="" and entry.get("Class")==""):
        if(listOfEntry!=[]):
            #print(listOfEntry)
            doclist += entrylist
            continue
    elif(entry.get("Commodity")==""):
        continue
    else:
        if(entry.get("Class Definition")!=""):
            f = str(sent_tokenize(entry.get("Class Definition").lower())).strip('[]').strip('\'')
            #print(f)
        else:
            f=str([" "]).strip('[]').strip('\'')
        e = str(sent_tokenize(entry.get("Class Title").lower())).strip('[]').strip('\'')
        #print(e)
        g = str(sent_tokenize(entry.get("Commodity Title").lower())).strip('[]').strip('\'')
        #print(g)
        if(entry.get("Commodity Definition")!=""):
            h = str(sent_tokenize(entry.get("Commodity Definition").lower())).strip('[]').strip('\'')
            #print(h)
        else:
            h = str([" "]).strip('[]').strip('\'')
        i=int(entry.get("Commodity"))
        df7 = df7.append({'class':e+f,'commodity':g+h,'level_1': i},ignore_index=True)
        
  
        
doclist.append(listOfEntry) 

print("done")        
"""        
unspec_df = pd.read_excel('ignore/UNSPSC English v220601 project.xlsx','Sheet1', index_col=None, na_values=['NA'])
unspec = unspec_df.drop(['Family Definition','Family','Family Title','Segment Title','Segment Definition','Segment','Version','Key','Synonym','Acronym'],axis = 1)
cols = ['Class Title','Class Definition']
cols2 =['Commodity Title','Commodity Definition']
unspec["class"] = unspec[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
unspec["commodity"] = unspec[cols2].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
unspec2 = unspec.drop(['Class','Class Title','Class Definition','Commodity Title','Commodity Definition'],axis=1)
unspec2=unspec2.rename(columns={"Commodity":"level_1"})
nums = unspec2['level_1'].tolist()

In [3]:
county_df = pd.read_excel('ignore/eCAPS_COMM_11072019.xlsx','COMM_CLS', index_col=None, na_values=['NA'])
county_df2 = pd.read_excel('ignore/eCAPS_COMM_11072019.xlsx','COMM_ITM', index_col=None, na_values=['NA'])
county_df.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
county_df2.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
countyresult = pd.merge(county_df,county_df2,on='COMM_CLS')
cr = countyresult.drop(['DSCR_EXT_y','DSCR_EXT_DV_x','DSCR_EXT_x','COMM_DSCR_UP_x','COMM_CD_x','COMM_ITM_x','COMM_GP_x','COMM_DET_x','ACT_INACT_FL_x','NAICS_CD_x','COMM_DET_y','COMM_GP_y','COMM_DET_y','ACT_INACT_FL_y','NAICS_CD_y','COMM_DSCR_UP_y','DSCR_EXT_DV_y'],axis=1)
cr2 = cr.rename(columns={"KEYWD_x":"class","KEYWD_y":"commodity","COMM_CD_y":"level_1"})


In [4]:

df1= cr2[['class','commodity','level_1']]
df2 = df1.append(unspec2,ignore_index=True,sort=False)


In [5]:
df2.head(5)

,class,commodity,level_1
0,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",RADIO-NAVIGATION-AIRCRAFT,3510
1,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",RADAR-AIRCRAFT,3515
2,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",AIRPLANES,3520
3,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",AUTOMATIC PILOT SYSTEMS,3525
4,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",BAGGAGE HANDLING EQUIPMENT AND PARTS,3526


In [6]:
df9 = df2.apply(lambda x: ','.join(x.astype(str)),axis = 1)

In [7]:
df_clean2 = pd.DataFrame({'clean':df9})

In [8]:
sent23 = [row.split(',') for row in df_clean2['clean']]

In [9]:
print(len(sent23))

163115


In [24]:
model2 = gensim.models.word2vec.Word2Vec(sent23,min_count=1,size=300, workers=6, window= 3,cbow_mean=1)

In [25]:
model2.wv.__getitem__('39101601')

array([ 6.81956531e-04,  1.25049474e-03, -9.41170583e-05, -3.63043684e-04,
        1.63510826e-03, -2.95380829e-04,  1.34745555e-03,  6.50337955e-04,
       -5.11979859e-04, -3.34227749e-04, -5.52599144e-04,  1.14899781e-03,
       -4.94596199e-04, -7.02638878e-04, -1.46510210e-04, -6.29882386e-04,
       -4.82608593e-04, -2.16885051e-03,  1.46272383e-03, -2.00267788e-03,
       -4.31637018e-04, -1.67274592e-03, -2.69638095e-03, -3.91958223e-04,
        1.63727236e-04,  9.04066837e-04, -4.44173871e-04,  1.64567516e-03,
       -5.87945106e-04, -1.01611449e-03,  6.04745932e-04, -2.04383978e-03,
        5.06551587e-05,  2.32606195e-03, -3.00471997e-03,  9.43856896e-04,
        3.33012664e-04, -6.96234056e-04,  1.61731057e-03, -1.61609962e-03,
       -7.71443592e-04,  1.97780086e-03,  1.59876526e-03, -9.89899971e-04,
        1.65839819e-03,  1.27749296e-03,  6.33954420e-04, -2.46457732e-03,
        3.19606764e-03,  8.84656765e-05, -1.98670034e-03,  2.15013023e-03,
       -4.44119563e-04,  

In [36]:
def NumberofDocs(li,nums):
    shortlist=[]
    for i in li:
        if(i!='nan'):
            point = model2.wv.similarity(str(i),nums)
            if(point>.9000001):
                shortlist.append(i) 
    
    return shortlist

result =NumberofDocs(nums,'4045') 
print(len(result))
print(result)

24271
[10101504, 10101516, 10101518, 10101603, 10101607, 10101703, 10101711, 10101719, 10101720, 10101725, 10101727, 10101729, 10101731, 10101803, 10101805, 10101808, 10101905, 10101906, 10102001, 10102002, 10102006, 10102011, 10102013, 10102017, 10102104, 10102106, 10102107, 10111301, 10121604, 10121703, 10121803, 10121805, 10121806, 10122002, 10122103, 10131506, 10131603, 10131701, 10141505, 10141610, 10141612, 10151505, 10151508, 10151517, 10151518, 10151525, 10151542, 10151543, 10151544, 10151603, 10151605, 10151612, 10151702, 10151704, 10151705, 10151706, 10151707, 10151805, 10151904, 10151905, 10151925, 10151927, 10152008, 10152012, 10152032, 10152034, 10152037, 10152058, 10152059, 10152103, 10152202, 10161505, 10161511, 10161517, 10161519, 10161520, 10161530, 10161531, 10161532, 10161535, 10161541, 10161544, 10161547, 10161549, 10161551, 10161555, 10161570, 10161573, 10161575, 10161576, 10161579, 10161580, 10161585, 10161586, 10161587, 10161602, 10161606, 10161609, 10161611, 101

In [33]:
result2 =NumberofDocs(nums,'3526') 
print(len(result2))
print(result2)

73
[10302422, 10316103, 10402147, 10415701, 20122851, 23153205, 24111817, 24141707, 27112221, 31401702, 40182905, 41141966, 41161506, 50391554, 50391632, 50394902, 50422704, 50442003, 51101634, 51112511, 51201827, 51204203, 51273019, 51453405, 51461525, 52131503, 56121501, 60121604, 72141204, 72141510, 77121601, 81151602, 85223205, 85335703, 85351604, 85356005, 85422405, 85452705, 85493037, 85493748, 85494745, 85495003, 85504202, 85504417, 85511052, 85515862, 85532448, 85582310, 85591454, 85601304, 85642508, 85652004, 85662247, 85671226, 85672505, 85694643, 85702373, 85711131, 85722114, 85723502, 85724055, 85731126, 85752254, 85753120, 85753542, 85761029, 85772323, 85772328, 85781909, 85874881, 86121502, 92112001, 95101703]


In [34]:
model2.wv.similarity('39101601','39101603')

0.625332

In [35]:
model2.wv.similarity('10101708','4042')

0.65047294

In [31]:
model2.wv.most_similar('3105')[:15]

[(' (COMPLEX REGIONAL PAIN SYNDROME I of UPPER LIMB G90.51)',
  0.9684817790985107),
 ('Drainage of right parotid gland with drainage device', 0.9683722257614136),
 ('Diagnosis of tuberculosis of lung', 0.968213677406311),
 (' BILATERAL H34.13)', 0.9682079553604126),
 (' (ABSCESS of SPLEEN D73.3)', 0.968127429485321),
 ("Diagnoses of combined immunodeficiencies This classification denotes the group of activities that yields a diagnosis of Combined Immunodeficiencies. Combined Immunodeficiencies is listed In The World Health Organization's International Statistical Classification of Diseases and Related Health Problems or Icd-10 as a Three Digit Category under code A00. Icd-10 Also Has The Following Combined Immunodeficiencies Children in its Four-Character Category: D81.0 or Severe Combined Immunodeficiency or Scid With Reticular Dysgenesis; D81.1 or Severe Combined Immunodeficiency or Scid With Low T- and B-Cell Numbers; D81.2 or Severe Combined Immunodeficiency or Scid With Low Or No

In [17]:
"""
model2.wv.vocab
model2.save("W2V-Model")

text = "AIR CONDITIONING, HEATING, AND VENTILATING: EQUIPMENT, PARTS  HYDRONIC SPECIALTIES"
idt =['101'] 
text = text.lower()
newEntry = []
newEn =[]
stop_words = set(stopwords.words('english'))
ch = [ '``','``', "''",',','.','\\n',"'",";",":","(",")","-","--"]


Wtest = word_tokenize(text)
Wtest = [ j for j in Wtest if not j in ch ]
Wtest = [w for w in Wtest if not w in stop_words]
newEntry.append(Wtest)

def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element+" ")
    return result

newEntry[0] = concatenate_list_data(newEntry[0])
newEntry += idt
newEn.append(newEntry)
print(newEn)

new_mod = gensim.models.Word2Vec.load("W2V-Model")
new_mod.build_vocab(newEn,update = True)
new_mod.train(newEn,total_examples=1,epochs = 1)

new_mod.wv.similarity('30121703.0','101')
new_mod.wv.most_similar_cosmul('101')

new_mod.wv.similar_by_vector('101')

"""

'\nmodel2.wv.vocab\nmodel2.save("W2V-Model")\n\ntext = "AIR CONDITIONING, HEATING, AND VENTILATING: EQUIPMENT, PARTS  HYDRONIC SPECIALTIES"\nidt =[\'101\'] \ntext = text.lower()\nnewEntry = []\nnewEn =[]\nstop_words = set(stopwords.words(\'english\'))\nch = [ \'``\',\'``\', "\'\'",\',\',\'.\',\'\\n\',"\'",";",":","(",")","-","--"]\n\n\nWtest = word_tokenize(text)\nWtest = [ j for j in Wtest if not j in ch ]\nWtest = [w for w in Wtest if not w in stop_words]\nnewEntry.append(Wtest)\n\ndef concatenate_list_data(list):\n    result= \'\'\n    for element in list:\n        result += str(element+" ")\n    return result\n\nnewEntry[0] = concatenate_list_data(newEntry[0])\nnewEntry += idt\nnewEn.append(newEntry)\nprint(newEn)\n\nnew_mod = gensim.models.Word2Vec.load("W2V-Model")\nnew_mod.build_vocab(newEn,update = True)\nnew_mod.train(newEn,total_examples=1,epochs = 1)\n\nnew_mod.wv.similarity(\'30121703.0\',\'101\')\nnew_mod.wv.most_similar_cosmul(\'101\')\n\nnew_mod.wv.similar_by_vector(\'10